In [4]:
import pandas as pd

# 月份对应的延误概率（来自你已有模型输出） - 注意是 0~1 的概率
monthly_delay_probs = {
    1: 75 / (1803 + 77),     
    2: 210 / (1766 + 203),   
    3: 275 / (2010 + 287),   
    4: 52 / (2095 + 73),     
    5: 83 / (2384 + 100),   
    6: 22 / (2185 + 26),    
    7: 49 / (2335 + 64),    
    8: 46 / (2414 + 79),    
    9: 26 / (970 + 39),     
    10: 23 / (1133 + 35),    
    11: 181 / (1584 + 194),  
    12: 65 / (1229 + 66),   
}


# 假设一个虚拟订单的平均运费（可替换为你的真实订单均值）
average_shipping_cost = 10  # 单位：R$

# 投保率
coverage_rate = 0.5  # 一半人投保

# 定价逻辑：延误成本 × 安全系数（1.2） ÷ 投保率（0.5）
insurance_prices = {
    month: round(delay_prob * average_shipping_cost * 1.2 / coverage_rate, 2)
    for month, delay_prob in monthly_delay_probs.items()
}

# 输出结果
print("If average_shipping_cost = 100, Monthly Insurance Suggested Pricing (in R$):\n")
for month in range(1, 13):
    prob = monthly_delay_probs[month]
    price = insurance_prices[month]
    print(f"Month {month}: late rate {prob:.2%}, Insurance Pricing Proposal: R${price}")


If average_shipping_cost = 100, Monthly Insurance Suggested Pricing (in R$):

Month 1: late rate 3.99%, Insurance Pricing Proposal: R$0.96
Month 2: late rate 10.67%, Insurance Pricing Proposal: R$2.56
Month 3: late rate 11.97%, Insurance Pricing Proposal: R$2.87
Month 4: late rate 2.40%, Insurance Pricing Proposal: R$0.58
Month 5: late rate 3.34%, Insurance Pricing Proposal: R$0.8
Month 6: late rate 1.00%, Insurance Pricing Proposal: R$0.24
Month 7: late rate 2.04%, Insurance Pricing Proposal: R$0.49
Month 8: late rate 1.85%, Insurance Pricing Proposal: R$0.44
Month 9: late rate 2.58%, Insurance Pricing Proposal: R$0.62
Month 10: late rate 1.97%, Insurance Pricing Proposal: R$0.47
Month 11: late rate 10.18%, Insurance Pricing Proposal: R$2.44
Month 12: late rate 5.02%, Insurance Pricing Proposal: R$1.2


In [2]:
import pandas as pd

# 载入数据（你已有的清洗后表）
df = pd.read_csv("../CleanDeta/order_full_cleaned.csv", parse_dates=['order_purchase_timestamp'])

# 添加月份
df['month'] = df['order_purchase_timestamp'].dt.month

# 模型得到的每月延误率（来自你的 confusion matrix）
monthly_delay_probs = {
    1: 75 / (1803 + 77),     
    2: 210 / (1766 + 203),   
    3: 275 / (2010 + 287),   
    4: 52 / (2095 + 73),     
    5: 83 / (2384 + 100),   
    6: 22 / (2185 + 26),    
    7: 49 / (2335 + 64),    
    8: 46 / (2414 + 79),    
    9: 26 / (970 + 39),     
    10: 23 / (1133 + 35),    
    11: 181 / (1584 + 194),  
    12: 65 / (1229 + 66),   
}

# 参数
safety_margin = 1.2   # 安全系数：你想要赚的钱倍数
coverage_rate = 0.5   # 只有一半人投保

# 只保留需要的字段
df = df[['order_id', 'month', 'payment_value']].copy()

# 映射每月的延误率
df['delay_prob'] = df['month'].map(monthly_delay_probs)

# 计算每笔订单的保险价格（核心公式）
df['insurance_price'] = round(df['delay_prob'] * df['payment_value'] * safety_margin / coverage_rate, 2)

# 展示前几行
print(df.head(20))


                            order_id  month  payment_value  delay_prob  \
0   e481f51cbdc54678b7cc49136f2d6af7     10          18.12    0.019692   
1   e481f51cbdc54678b7cc49136f2d6af7     10           2.00    0.019692   
2   e481f51cbdc54678b7cc49136f2d6af7     10          18.59    0.019692   
3   53cdb2fc8bc7dce0b6741e2150273451      7         141.46    0.020425   
4   47770eb9100c2d0c44946d9cf07ec65d      8         179.12    0.018452   
5   949d5b44dbf5de918fe9c16f97b45f8a     11          72.20    0.101800   
6   ad21c59c0840e6cb83a9ceb5573f8159      2          28.62    0.106653   
7   a4591c265e18cb1dcee52889e2d8acc3      7         175.26    0.020425   
8   6514b8ad8028c9f2cc2374ded245783f      5          75.16    0.033414   
9   76c6e866289321a7c93b82b54852dc33      1          35.95    0.039894   
10  e69bfb5eb88e0ed6a785585b27e16dbf      7         161.42    0.020425   
11  e69bfb5eb88e0ed6a785585b27e16dbf      7           8.34    0.020425   
12  e6ce16cb79ec1d90b1da9085a6118aeb  

In [3]:
import pandas as pd

# 载入数据（你已有的清洗后表）
df = pd.read_csv("../CleanDeta/order_full_cleaned.csv", parse_dates=['order_purchase_timestamp'])

# 添加月份
df['month'] = df['order_purchase_timestamp'].dt.month

# 模型得到的每月延误率（来自你的 confusion matrix）
monthly_delay_probs = {
    1: 75 / (1803 + 77),     
    2: 210 / (1766 + 203),   
    3: 275 / (2010 + 287),   
    4: 52 / (2095 + 73),     
    5: 83 / (2384 + 100),   
    6: 22 / (2185 + 26),    
    7: 49 / (2335 + 64),    
    8: 46 / (2414 + 79),    
    9: 26 / (970 + 39),     
    10: 23 / (1133 + 35),    
    11: 181 / (1584 + 194),  
    12: 65 / (1229 + 66),   
}
# 参数
safety_margin = 1.2   # 安全系数：你希望从保险中赚的钱倍数
coverage_rate = 0.5   # 只有一半人投保

# 只保留需要的字段（注意换成 freight_value）
df = df[['order_id', 'month', 'freight_value']].copy()

# 映射每月的延误率
df['delay_prob'] = df['month'].map(monthly_delay_probs)

# 计算每笔订单的保险价格（基于运费而非支付金额）
df['insurance_price'] = round(df['delay_prob'] * df['freight_value'] * safety_margin / coverage_rate, 2)

# 展示前20行
print(df.head(30))


                            order_id  month  freight_value  delay_prob  \
0   e481f51cbdc54678b7cc49136f2d6af7     10           8.72    0.019692   
1   e481f51cbdc54678b7cc49136f2d6af7     10           8.72    0.019692   
2   e481f51cbdc54678b7cc49136f2d6af7     10           8.72    0.019692   
3   53cdb2fc8bc7dce0b6741e2150273451      7          22.76    0.020425   
4   47770eb9100c2d0c44946d9cf07ec65d      8          19.22    0.018452   
5   949d5b44dbf5de918fe9c16f97b45f8a     11          27.20    0.101800   
6   ad21c59c0840e6cb83a9ceb5573f8159      2           8.72    0.106653   
7   a4591c265e18cb1dcee52889e2d8acc3      7          27.36    0.020425   
8   6514b8ad8028c9f2cc2374ded245783f      5          15.17    0.033414   
9   76c6e866289321a7c93b82b54852dc33      1          16.05    0.039894   
10  e69bfb5eb88e0ed6a785585b27e16dbf      7          19.77    0.020425   
11  e69bfb5eb88e0ed6a785585b27e16dbf      7          19.77    0.020425   
12  e6ce16cb79ec1d90b1da9085a6118aeb  